[ Dataset Information ]

1. train.csv
- train data: Price data of distributed items from 01/01/2019 to 03/03/2023
- item code
  TG: Tangerine, BC: Broccoli, RD: Radish, CR: Carrot, CB: Cabbage
  corporation: Distribution corporation code, ranging from corporation A to F
  location: Location code -> J: Jeju, S: Seogwipo
  supply(kg): Distributed quantity in kilograms
  price(원/kg) : Price per kilogram of the distributed items in Korean Won

2. international_trade.csv
- international trade information: Import and export information for related items

3. test.csv
- test data: Data from 03/04/2023 to 03/31/2023

4. sample_submission.csv
- submission format: Predict price(won/kg) from 03/04/2023 to 03/31/2023
- ID: Identifier consisting of item, corporation and location codes
- must fill in the predicted price(won/kg) in the answer column for the corresponding ID

In [5]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import sys
import holidays

import sklearn
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import xgboost
from xgboost import XGBRegressor
import catboost
from catboost import CatBoostRegressor

In [43]:
print(f"python version : {sys.version}")
print(f"pandas version : {pd.__version__}")
print(f"numpy version : {np.__version__}")
print(f"sklearn version : {sklearn.__version__}")
print(f"xgboost version : {xgboost.__version__}")
print(f"catboost version : {catboost.__version__}")

python version : 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:07:17) [Clang 14.0.6 ]
pandas version : 2.2.2
numpy version : 1.26.4
sklearn version : 1.4.2
xgboost version : 2.1.1
catboost version : 1.2.5


In [45]:
#Ensure reproductibility of results
def seed_everything(seed: int = 2024):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(2024)

# 0. Load Data

In [48]:
train = pd.read_csv("/Users/jeonseungmi/Projects/lspp/lspp_train.csv")
test = pd.read_csv("/Users/jeonseungmi/Projects/lspp/lspp_test.csv")
train

,ID,timestamp,item,corporation,location,supply(kg),price(won/kg)
0,TG_A_J_20190101,1/1/19,TG,A,J,0.0,0
1,TG_A_J_20190102,1/2/19,TG,A,J,0.0,0
2,TG_A_J_20190103,1/3/19,TG,A,J,60601.0,1728
3,TG_A_J_20190104,1/4/19,TG,A,J,25000.0,1408
4,TG_A_J_20190105,1/5/19,TG,A,J,32352.0,1250
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2/27/23,RD,F,J,452440.0,468
59393,RD_F_J_20230228,2/28/23,RD,F,J,421980.0,531
59394,RD_F_J_20230301,3/1/23,RD,F,J,382980.0,574
59395,RD_F_J_20230302,3/2/23,RD,F,J,477220.0,523


# 1. Preprocessing

In [51]:
def pre_all(train, test):
    print(f"train size before preprocessing : {train.shape}")
    print(f"test size before preprocessing : {test.shape}")
    print("================= Preprocessing =================")

    #Merge and Preprocess
    train["timestamp"] = pd.to_datetime(train["timestamp"])
    test["timestamp"] = pd.to_datetime(test["timestamp"])
    df = pd.concat([train,test]).reset_index(drop = True)

    df.rename(columns={'supply(kg)':'supply', 'price(won/kg)':'price'},inplace=True)

    #Add year/month/day
    df['year']=df['timestamp'].dt.year
    df['month']=df['timestamp'].dt.month
    df['day']=df['timestamp'].dt.day

    #Add weekday
    df['week_day']=df['timestamp'].dt.weekday

    #Add month-year variable: type of month-year, cumulative values by month
    le = LabelEncoder()
    df["month_year"] = df["timestamp"].map(lambda x :str(x.month) + "-" + str(x.year))

    #Label incoding
    df["month_year"] = le.fit_transform(df["month_year"])


    #Add week variable
    df["week"] = df["timestamp"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

    #cumulative values by week
    week_list=[]
    for i in range(len(df['year'])) :
        if df['year'][i] == 2019 :
            week_list.append(int(df['week'][i]))
        elif df['year'][i] == 2020 :
            week_list.append(int(df['week'][i])+52)
        elif df['year'][i] == 2021 :
            week_list.append(int(df['week'][i])+52+53)
        elif df['year'][i] == 2022 :
            week_list.append(int(df['week'][i])+52+53+53)
        elif df['year'][i] == 2023 :
            week_list.append(int(df['week'][i])+52+53+53+52)
    df['week_num']= week_list

    #Discovered that the last week of December 2019 is being classified as the first week of the new year and made adjustments
    df.loc[df['timestamp']=='2019-12-30','week_num']=52
    df.loc[df['timestamp']=='2019-12-31','week_num']=52


    #Add holidays variable
    def make_holi(x):
        kr_holi = holidays.KR()

        if x in kr_holi:
            return 1
        else:
            return 0
        
    df["holiday"] = df["timestamp"].map(lambda x : make_holi(x))
    

    #Split train, test
    train = df[~df["price"].isnull()].sort_values("timestamp").reset_index(drop = True)
    test = df[df["price"].isnull()].sort_values("timestamp").reset_index(drop=True)


    print(f"train size after preprocessing : {train.shape}")
    print(f"test size after preprocessing : {test.shape}")

    return train, test

In [53]:
train_pre, test_pre = pre_all(train, test)

train size before preprocessing : (59397, 7)
test size before preprocessing : (1092, 5)
================= Preprocessing =================


/var/folders/4z/mczsf4vs4wsgt6fj97fl9lxw0000gn/T/ipykernel_33555/2443573591.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train["timestamp"] = pd.to_datetime(train["timestamp"])


train size after preprocessing : (59397, 15)
test size after preprocessing : (1092, 15)


In [56]:
train_pre

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,month_year,week,week_num,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,1,1,1
1,CB_A_S_20190101,2019-01-01,CB,A,S,0.0,0.0,2019,1,1,1,0,1,1,1
2,RD_D_J_20190101,2019-01-01,RD,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
3,BC_D_J_20190101,2019-01-01,BC,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
4,CB_F_J_20190101,2019-01-01,CB,F,J,0.0,0.0,2019,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,CR_E_S_20230303,2023-03-03,CR,E,S,0.0,0.0,2023,3,3,4,26,9,219,0
59393,BC_A_S_20230303,2023-03-03,BC,A,S,3776.0,2875.0,2023,3,3,4,26,9,219,0
59394,CB_E_J_20230303,2023-03-03,CB,E,J,0.0,0.0,2023,3,3,4,26,9,219,0
59395,BC_D_J_20230303,2023-03-03,BC,D,J,1776.0,3059.0,2023,3,3,4,26,9,219,0


In [55]:
test_pre

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,month_year,week,week_num,holiday
0,TG_A_J_20230304,2023-03-04,TG,A,J,NaN,NaN,2023,3,4,5,26,9,219,0
1,TG_E_S_20230304,2023-03-04,TG,E,S,NaN,NaN,2023,3,4,5,26,9,219,0
2,BC_B_J_20230304,2023-03-04,BC,B,J,NaN,NaN,2023,3,4,5,26,9,219,0
3,TG_E_J_20230304,2023-03-04,TG,E,J,NaN,NaN,2023,3,4,5,26,9,219,0
4,BC_B_S_20230304,2023-03-04,BC,B,S,NaN,NaN,2023,3,4,5,26,9,219,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,TG_A_J_20230331,2023-03-31,TG,A,J,NaN,NaN,2023,3,31,4,26,13,223,0
1088,RD_D_J_20230331,2023-03-31,RD,D,J,NaN,NaN,2023,3,31,4,26,13,223,0
1089,CR_D_J_20230331,2023-03-31,CR,D,J,NaN,NaN,2023,3,31,4,26,13,223,0
1090,TG_E_J_20230331,2023-03-31,TG,E,J,NaN,NaN,2023,3,31,4,26,13,223,0


# 2. Products excluding TG (Tangerine)

## 2-1. Preprocessing

In [60]:
#Remove extreme outliers 
tg_idx = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]>20000)].index
rd_idx = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]>5000)].index
bc_idx = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]>8000)].index
cb_idx = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]>2300)].index

train_pre.loc[tg_idx,"price"] = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[rd_idx,"price"] = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[bc_idx,"price"] = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[cb_idx,"price"] = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]!=0)]["price"].mean()


#Non TG items 
print(f"train column : {train_pre.columns}")
print(f"test column : {test_pre.columns}")

train_notg = train_pre[train_pre["item"] !="TG"]
test_notg = test_pre[test_pre["item"] != "TG"]


#Incoding
Xy = pd.get_dummies(train_notg.sort_values(by = ["timestamp"]).reset_index(drop=True).drop(columns = ["supply"]), columns = ["item","corporation","location"])
answer_notg = pd.get_dummies(test_notg.drop(columns = ["timestamp","supply","price"]), columns = [ "item","corporation","location"])

print(Xy.columns)

train column : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'month_year', 'week', 'week_num',
       'holiday'],
      dtype='object')
test column : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'month_year', 'week', 'week_num',
       'holiday'],
      dtype='object')
Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'month_year', 'week', 'week_num', 'holiday', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S'],
      dtype='object')


## 2-2. Modeling and Training Prediction

In [63]:
#Define ensemble models
cat = CatBoostRegressor(random_state = 2024, 
                            n_estimators = 1000, 
                            learning_rate = 0.01, 
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)
xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators = [("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_notg.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx] < 0:
        pred[idx] = 0
answer_notg["answer"] = pred

answer_notg[["ID","answer"]]

0:	learn: 934.5536226	total: 7.87ms	remaining: 7.87s
999:	learn: 373.4796858	total: 4.26s	remaining: 0us


,ID,answer
2,BC_B_J_20230304,2434.067720
4,BC_B_S_20230304,146.336324
6,BC_C_J_20230304,2282.493562
7,BC_A_S_20230304,2870.143958
10,BC_D_J_20230304,2836.797660
...,...,...
1085,RD_D_S_20230331,431.965819
1086,CR_C_J_20230331,1757.552641
1088,RD_D_J_20230331,341.042890
1089,CR_D_J_20230331,1855.161714


# 3. TG (1) (Tangerine)

## 3-1. Preprocessing

In [68]:
train_pre, test_pre = pre_all(train, test)

#Exclude holidays that are not non-working days
no_holi = list(train_pre[(train_pre["item"] =="TG") & (train_pre["holiday"]==1) & (train_pre["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])
noholi_idx = train_pre[train_pre["timestamp"].isin(no_holi)]["holiday"].index
for idx in noholi_idx:
    train_pre.loc[idx, "holiday"] = 0

#Sort train and test data in chronological order
train_tg = train_pre[train_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop = True)
test_tg = test_pre[test_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop = True)

Xy = pd.get_dummies(train_tg, columns = [ "item","corporation","location"]).drop(columns = ["supply"])
answer_tg1 = pd.get_dummies(test_tg, columns = [ "item","corporation","location"]).drop(columns = ["timestamp","supply","price"])
print(f"train column : {Xy.columns}")
print(f"test column : {answer_tg1.columns}")
Xy["price"] = np.sqrt(Xy["price"])

train size before preprocessing : (59397, 7)
test size before preprocessing : (1092, 5)
================= Preprocessing =================
train size after preprocessing : (59397, 15)
test size after preprocessing : (1092, 15)
train column : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'month_year', 'week', 'week_num', 'holiday', 'item_TG', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')
test column : Index(['ID', 'year', 'month', 'day', 'week_day', 'month_year', 'week',
       'week_num', 'holiday', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'location_J',
       'location_S'],
      dtype='object')


## 3-2. Modeling and Training Prediction

In [70]:
#Define models
cat = CatBoostRegressor(random_state = 2024, 
                            n_estimators = 1000, 
                            learning_rate = 0.01, 
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


#voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_tg1.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_tg1["answer"] = np.power(pred,2)

answer_tg1[["ID","answer"]]

0:	learn: 30.9647639	total: 5.03ms	remaining: 5.02s
999:	learn: 15.4226569	total: 2.82s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2651.246149
1,TG_E_S_20230304,3461.125793
2,TG_E_J_20230304,853.696182
3,TG_D_S_20230304,2855.485377
4,TG_D_J_20230304,616.626262
...,...,...
275,TG_D_J_20230331,1920.181445
276,TG_D_S_20230331,4934.844712
277,TG_A_S_20230331,5283.210531
278,TG_E_S_20230331,4089.912211


# 4. TG (2) (Generalization)

- Additional modeling for generalization

## 4-1. Preprocessing

In [79]:
train_tg2 = train_pre[train_pre["item"] == "TG"]
test_tg2 = test_pre[test_pre["item"] == "TG"]

Xy2 = pd.get_dummies(train_tg2.sort_values(by = ["timestamp", "corporation","location"]).reset_index(drop=True).drop(columns = ["item","supply"]), columns = [ "corporation","location"])
answer_tg2 = pd.get_dummies(test_tg2.drop(columns = ["timestamp","supply","price","item"]), columns = [ "corporation","location"])
print(Xy2.columns)


#Route of dependent variable
Xy2["price"] = np.sqrt(Xy2["price"])

Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'month_year', 'week', 'week_num', 'holiday', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')


## 4-2. Modeling and Training Prediction

In [81]:
#Define models and predict training
n_estimators =1000
lrs = 0.05
max_depths = 10
l2_leaf_reg = 3

cat = CatBoostRegressor(random_state = 2024, 
                                n_estimators = n_estimators, 
                                learning_rate = lrs, 
                                depth = max_depths, 
                                l2_leaf_reg = l2_leaf_reg,
                                metric_period = 1000)

cat.fit(Xy2.drop(columns = ["timestamp", "ID","price"]), Xy2["price"])

pred2 = cat.predict(answer_tg2.drop(columns = ["ID"]))
for idx in range(len(pred2)):
    if pred2[idx]<0:
        pred2[idx]= 0
answer_tg2["answer"] = np.power(pred2,2)

answer_tg2[["ID","answer"]]

0:	learn: 30.2729298	total: 6.06ms	remaining: 6.05s
999:	learn: 10.9417677	total: 2.85s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,3476.160195
1,TG_E_S_20230304,3548.749087
3,TG_E_J_20230304,776.932347
5,TG_D_S_20230304,3703.939980
8,TG_D_J_20230304,156.821244
...,...,...
1074,TG_D_S_20230331,4116.318156
1077,TG_A_S_20230331,5387.117921
1079,TG_E_S_20230331,4147.688534
1087,TG_A_J_20230331,6900.283025


# 5. TG Ensemble

In [84]:
total1 = pd.concat([answer_tg1[["ID","answer"]],answer_notg[["ID","answer"]]])
total2 = pd.concat([answer_tg2[["ID","answer"]],answer_notg[["ID","answer"]]])

In [86]:
#TG Ensemble (Average)
df = pd.merge(total1, total2, how = "inner", on="ID")
df["answer"] = (df["answer_x"]+df["answer_y"])/2
df["item"] = df["ID"].map(lambda x :x.split("_")[0])

# 6. Post-processing

In [89]:
#Identify total minimum and minimum of March
df.loc[(df['item']=='TG')&(df['answer']<400),'answer'] = 0  #T: 551, 3월: 675
df.loc[(df['item']=='CB')&(df['answer']<50),'answer'] = 0  #T: 162, 3월: 200
df.loc[(df['item']=='RD')&(df['answer']<10),'answer'] = 0  #T: 50, 3월: 124
df.loc[(df['item']=='CR')&(df['answer']<150),'answer'] = 0  #T: 250, 3월: 450
df.loc[(df['item']=='BC')&(df['answer']<100),'answer'] = 0  #T: 205, 3월: 205.0

In [91]:
df = df.drop(columns = ["answer_x","answer_y", "item"])
df

,ID,answer
0,TG_A_J_20230304,3063.703172
1,TG_E_S_20230304,3504.937440
2,TG_E_J_20230304,815.314264
3,TG_D_S_20230304,3279.712679
4,TG_D_J_20230304,0.000000
...,...,...
1087,RD_D_S_20230331,431.965819
1088,CR_C_J_20230331,1757.552641
1089,RD_D_J_20230331,341.042890
1090,CR_D_J_20230331,1855.161714


In [95]:
#Submission file
df.to_csv('answer.csv',index = False)